In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import stats
import glob
import os
from natsort import natsorted
plt.rcParams['figure.figsize'] = [10, 5]

In [9]:
path = 'viz_predictions/new'
savepath = path+'/detectionX'

if not os.path.exists(savepath):
    os.makedirs(savepath)

In [10]:
#
for image in natsorted(glob.glob(path+'/rgb/'+'*.jpg')):
    im_num = os.path.splitext(os.path.basename(image))[0]
    img = cv2.imread(path+'/gray/'+os.path.split(image)[1], 0)
    img_rgb = cv2.imread(path+'/rgb/'+os.path.split(image)[1], 1)
    keypoints = np.load(path+'/keypoints/'+im_num+'.npy')
    pose = np.load(path+'/pose/'+im_num+'.npy') #pose keypoint numpy array

    
    handloc = keypoints[:,:,:,0:2].astype(int)
    hands3D = handloc
    
    
    #Dictionary of keypoinnt and index
    part_loc = {'left':0, 'right':1, 'right_elbow':3, 'right_wrist':4, 'left_elbow':6, 'left_wrist':7}
    
    #Extracting depth values of each hand named as 'person_number__left/right'
    hands3D = np.zeros(keypoints.shape)
    num_ppl = keypoints.shape[1]

    for pers in range(num_ppl):
        for hand in range(hands3D.shape[0]):
            depth = img[handloc[hand,pers,:,1], handloc[hand,pers,:,0]] 
            hands3D[hand,pers,:,2] = depth
        hands3D[:,pers,:,0:2] = handloc[:,pers,:,:]
        hands3D = hands3D.astype(int)
    
    #Adjust hadnkeypoint depth using elbow and wrist depth values
    hands3D_ = hands3D.copy()

    rightarm = pose[:, 3:4, 0:2].astype(int)
    leftarm = pose[:, 6:7, 0:2].astype(int)

    Rarm_depth = img[rightarm[:,:,1], rightarm[:,:,0]]
    Larm_depth = img[leftarm[:,:,1], leftarm[:,:,0]]
    Rdelta = Rarm_depth.mean(axis=1).astype(int).reshape(pose.shape[0],1)
    Ldelta = Larm_depth.mean(axis=1).astype(int).reshape(pose.shape[0],1)

    hands3D_[part_loc['left'],:,:,2] = (hands3D[part_loc['left'],:,:,2] + Ldelta)/2
    hands3D_[part_loc['right'],:,:,2] = (hands3D[part_loc['right'],:,:,2] + Rdelta)/2

    hands3D = hands3D_.copy()

    start_pt =[]
    end_pt = []

    for i in range(hands3D.shape[1]):
        for j in range(hands3D.shape[1]):
            if i != j:
                diff1 = abs(hands3D[0,i,:,:] - hands3D[0,j,:,:])
                eucdis1 = np.linalg.norm(diff1, axis=1)
                mode1, count1 = stats.mode(eucdis1)

                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis1) < 180 and sum(eucdis1) != 0:
#                     print('Handshake1 detected. RUN!!!')
                    xs1 = np.argmin(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1])))
                    ys1 = np.argmin(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0])))
                    xe1 = np.argmax(np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1])))
                    ye1 = np.argmax(np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0])))
                    start1 = (np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0]))[ys1], np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1]))[xs1])
                    end1 = (np.hstack((hands3D[0,i,:,0], hands3D[0,j,:,0]))[ye1], np.hstack((hands3D[0,i,:,1], hands3D[0,j,:,1]))[xe1])
                    start_pt.append(start1)
                    end_pt.append(end1)

                diff2 = abs(hands3D[0,i,:,:] - hands3D[1,j,:,:])
                eucdis2 = np.linalg.norm(diff2, axis=1)
                mode2, count2 = stats.mode(eucdis2)

                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis2) < 180 and sum(eucdis2) != 0:
#                     print('Handshake2 detected. RUN!!!')
                    xs2 = np.argmin(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1])))
                    ys2 = np.argmin(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0])))
                    xe2 = np.argmax(np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1])))
                    ye2 = np.argmax(np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0])))
                    start2 = (np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0]))[ys2], np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1]))[xs2])
                    end2 = (np.hstack((hands3D[0,i,:,0], hands3D[1,j,:,0]))[ye2], np.hstack((hands3D[0,i,:,1], hands3D[1,j,:,1]))[xe2])
                    start_pt.append(start2)
                    end_pt.append(end2)

                diff3 = abs(hands3D[1,i,:,:] - hands3D[0,j,:,:])
                eucdis3 = np.linalg.norm(diff3, axis=1)
                mode3, count3 = stats.mode(eucdis3)

                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis3) < 180 and sum(eucdis3) != 0:
#                     print('Handshake3 detected. RUN!!!')
                    xs3 = np.argmin(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1])))
                    ys3 = np.argmin(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0])))
                    xe3 = np.argmax(np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1])))
                    ye3 = np.argmax(np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0])))
                    start3 = (np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0]))[ys3], np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1]))[xs3])
                    end3 = (np.hstack((hands3D[1,i,:,0], hands3D[0,j,:,0]))[ye3], np.hstack((hands3D[1,i,:,1], hands3D[0,j,:,1]))[xe3])
                    start_pt.append(start3)
                    end_pt.append(end3)

                diff4 = abs(hands3D[1,i,:,:] - hands3D[1,j,:,:])
                eucdis4 = np.linalg.norm(diff4, axis=1)
                mode4, count4 = stats.mode(eucdis4)

                #Detect if Handshake and find min and max coordinates for bounding box and append to lisr
                if sum(eucdis4) < 180 and sum(eucdis4) != 0:
#                     print('Handshake4 detected. RUN!!!')
                    xs4 = np.argmin(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1])))
                    ys4 = np.argmin(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0])))
                    xe4 = np.argmax(np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1])))
                    ye4 = np.argmax(np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0])))
                    start4 = (np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0]))[ys4], np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1]))[xs4])
                    end4 = (np.hstack((hands3D[1,i,:,0], hands3D[1,j,:,0]))[ye4], np.hstack((hands3D[1,i,:,1], hands3D[1,j,:,1]))[xe4])
                    start_pt.append(start4)
                    end_pt.append(end4)


    #Plot bounding box
    for contact in range(len(start_pt)):
        cv2.rectangle(img_rgb, start_pt[contact], end_pt[contact],[0,0,255], 2)
#     plt.imshow(cv2.cvtColor(img_rgb,cv2.COLOR_BGR2RGB))
#     plt.show()
    cv2.imwrite(os.path.join(savepath, os.path.basename(image)), img_rgb)
    